In [4]:
pip install pandas beautifulsoup4 requests selenium webdriver-manager tqdm fuzzywuzzy

     |████████████████████████████████| 9.5 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 467 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 3.0 MB/s eta 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import selenium

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from tqdm import tqdm
from fuzzywuzzy import fuzz

/home/mabel/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
rotten = pd.read_csv('datos_totales.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datos_totales.csv'

In [ ]:
df_rotten=pd.DataFrame(rotten)

In [ ]:
lista_nombres_peli=df_rotten["1"].values
lista_años_peli=df_rotten["2"].values

In [ ]:
lista_nombre_año=list(zip(lista_nombres_peli, lista_años_peli))

In [ ]:
def puntuacion_rotten(lista_nombre_año):
    
    driver=webdriver.Chrome()
    driver.get("https://www.google.com")
    driver.maximize_window()
    sleep(1)

    #rechazar las cookies de google

    driver.find_element("css selector", "#L2AGLb").click()

    lista_tuplas_tomate=[]

    for pelicula in tqdm(lista_nombre_año):
        sleep(2)
        #Buscar en google nuestra pelicula
        driver.find_element("css selector", "#APjFqb").send_keys(f"{pelicula[0], pelicula[1]}, rotten tomatoes", Keys.ENTER)
        sleep(2)
        lista_pelicula_tomate=[]
        
        sleep(2)
    
        similitud = fuzz.ratio(pelicula[0], peli)
        
        try:
            #Guardamos en una variable el primer resultado que nos devuelve google para pdoer compararlo con nuestro título
            peli=driver.find_element("css selector", 'h3').text
            
            lista_pelicula_tomate.append(pelicula[0])
            
            if pelicula[0].lower() == peli.lower() or similitud >= 50:
                
                #Sacamos el texto de la puntuación
                
                try:
                    #Sacamos el texto de la puntuación y lo apendeamos
                    puntuacion_tomate=driver.find_element('css selector', '#rso > div:nth-child(1) > div > div > div:nth-child(3) > div > span:nth-child(2)').text
                    
                    lista_pelicula_tomate.append(puntuacion_tomate.replace("Valoración: ", ""))
                    
                except:
                    #Si está pero no tiene puntuación, apendeamos que no aparece puntuación
                    lista_pelicula_tomate.append("No aparece puntuación")
                    
                
            else:
                lista_pelicula_tomate.append("La pelicula no está en la página Rotten Tomatoes")
                
        except:
            pass
        
        lista_tuplas_tomate.append(tuple(lista_pelicula_tomate))
        
        driver.back()
        
    driver.quit()
    
    return lista_tuplas_tomate

In [ ]:
resultados_rotten=puntuacion_rotten()

df=pd.DataFrame(resultados_rotten, columns=["Nombre", "Puntuacion"])

df.head()
df.drop_duplicates()